<a id="1"></a>
# <div style="border: 2px solid #555; color:black; border-radius: 10px; background-color: #0074D9; padding: 10px; font-size: 20px; text-align: center;">Introduction</div>

**Table Of Content:**
* [Introduction](#1)
* [Refactor and Define utils](#2)
* [Refactor Train](#3)
* [Sweeps](#4)


<a id="2"></a>
# <div style="border: 2px solid #555; color:black; border-radius: 10px; background-color: #0074D9; padding: 10px; font-size: 20px; text-align: center;">Refactor and Define utils</div>
* [return top](#1)

In [1]:
%%writefile utils.py
import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
from datasets import Dataset
import io

def read_data():
    """
    return train , test
    """
    with open("/kaggle/working/artifacts/detect_llm_raw_data:v1/train_df.table.json") as json_data:
        data = json.load(json_data)
        train = pd.DataFrame(data = data["data"],columns=data["columns"])
        json_data.close()

    with open("/kaggle/working/artifacts/detect_llm_raw_data:v1/test_df.table.json") as json_data:
        data = json.load(json_data)
        test = pd.DataFrame(data = data["data"],columns=data["columns"])
        json_data.close()
    return train , test

def preprocess(train=None,test=None):
    """
    return dataset_train, dataset_test
    """
    train.fillna(" ",inplace=True)
    test.fillna(" ",inplace=True)
    train["text"] = train["Question"] + " " + train["Response"]
    test["text"] = test["Question"] + " " + test["Response"]
    df_train = train[["target","text"]]
    df_test = test[["text"]]
    dataset_train = Dataset.from_pandas(df_train)
    dataset_test = Dataset.from_pandas(df_test)
    
    return dataset_train, dataset_test


def dataset_tokenize_n_split(train, dataset_train, dataset_test,model_name):
    """
    return split_train_dataset,split_eval_dataset , tokenized_test , tokenizer
    """
    tokenizer       = AutoTokenizer.from_pretrained(model_name )
    def tokenize_function(examples):
    
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_train = dataset_train.map(tokenize_function, batched=True)
    tokenized_test  = dataset_test.map(tokenize_function, batched=True)
    tokenized_train = tokenized_train.remove_columns(['text'])
    tokenized_train = tokenized_train.rename_column("target", "labels")
    tokenized_test = tokenized_test.remove_columns(['text'])

    kf= StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
    for i , (tr_idx,val_idx) in enumerate(kf.split(train,train.target)):
        print(f"Fold : {i}")
        print(f"shape train : {tr_idx.shape}")
        print(f"shape val : {val_idx.shape}")
        break
        
    
    split_train_dataset = tokenized_train.select(tr_idx)
    split_eval_dataset = tokenized_train.select(val_idx)

    return split_train_dataset,split_eval_dataset , tokenized_test , tokenizer

def predict_fn(dataset_ = None):
    
    """
    return mean of all_probabilities (m,7)
    """
    input_ids = dataset_['input_ids']
    # token_type_ids = dataset_['token_type_ids']
    attention_mask = dataset_['attention_mask']

    # Move the input tensors to the GPU
    input_ids = torch.tensor(input_ids).to('cuda:0')
    # token_type_ids = torch.tensor(token_type_ids).to('cuda:0')
    attention_mask = torch.tensor(attention_mask).to('cuda:0')

    # Define batch size
    batch_size = 8

    # Calculate the number of batches
    num_samples = len(input_ids)
    num_batches = (num_samples + batch_size - 1) // batch_size

    # Initialize a list to store the softmax probabilities
    all_probabilities = []

    # Make predictions in batches
    with torch.no_grad():
        for batch in range(num_batches):
            start_idx = batch * batch_size
            end_idx = min((batch + 1) * batch_size, num_samples)

            batch_input_ids = input_ids[start_idx:end_idx]
    #         batch_token_type_ids = token_type_ids[start_idx:end_idx]
            batch_attention_mask = attention_mask[start_idx:end_idx]

            outputs = model(input_ids=batch_input_ids, 
    #                         token_type_ids=batch_token_type_ids, 
                            attention_mask=batch_attention_mask)
            logits = outputs.logits

            # Apply softmax to get probabilities
            probabilities = F.softmax(logits, dim=1)


            all_probabilities.extend(probabilities.tolist())
    return np.concatenate(all_probabilities,axis=0).reshape(dataset_.shape[0],7)


def conf_mat(df_val = None,preds_val = None):
    """
    no return
    """
    plt.figure(figsize=(8,8))
    ConfusionMatrixDisplay.from_predictions(df_val.target,np.argmax(preds_val,axis=1))
    plt.savefig(f"val_conf_matrix.png", format="png")
    plt.show();
    conf = wandb.Image(data_or_path="val_conf_matrix.png")
    wandb.log({"val_conf_matrix": conf})
def create_model(model_name = "distilroberta-base",num_labels = 7):
    """
    return
    """
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    # Specify the GPU device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Move your model to the GPU
    model.to(device);
    
    return model
    


Writing utils.py


<a id="3"></a>
# <div style="border: 2px solid #555; color:black; border-radius: 10px; background-color: #0074D9; padding: 10px; font-size: 20px; text-align: center;">Refactor Train</div>
* [return top](#1)

In [2]:
# %%writefile train.py
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
from datasets import Dataset
import json
from IPython.display import display
import wandb
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification,TrainerCallback
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import torch.nn.functional as F
from utils import *
import io

class WandbMetricsLogger(TrainerCallback):
    def on_evaluate(self, args, state, control, model, metrics):
        # Log metrics to Wandb
        wandb.log(metrics)
        
default_config = {
        'method': 'random',
        'metric': {
        'goal': 'minimize', 
        'name': 'eval_loss'
        },
    }


    # hyperparameters
parameters_dict = {
        'epochs': {
            'value': 2
            },
        'seed': {
            'value': 42
            },
        'batch_size': {
            'values': [4, 8, 16]
            },
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 1e-4,
            'max': 2e-3
        },
        'weight_decay': {
            'values': [0.0, 0.2]
        },
        'learning_sch': {
            'values': ['linear','polynomial','cosine']
        },
        'architecture': {
            'values': ["distilroberta-base","bert-base-uncased","distilbert-base-uncased"]
        },
    }


default_config['parameters'] = parameters_dict

def compute_metrics_fn(eval_preds):
    metrics = dict()

    # Extract the validation loss from eval_preds
    validation_loss = eval_preds.loss
    metrics['validation_loss'] = validation_loss

    return metrics

def parse_args():
    "Overriding default argments"
    argparser = argparse.ArgumentParser(description='Process hyper-parameters')
    argparser.add_argument('--batch_size', type=int, default=default_config.get("parameters").get("batch_size").get("values")[-1],
                           help='batch size')
    argparser.add_argument('--epochs', type=int, default=default_config.get("parameters").get("epochs").get("value"),
                           help='number of training epochs')
    argparser.add_argument('--lr', type=float, default=default_config.get("parameters").get("learning_rate").get("min"),
                           help='learning rate')
    argparser.add_argument('--seed', type=int, default=default_config.get("parameters").get("seed").get("value"),
                           help='random seed')
    argparser.add_argument('--weight_decay', type=float, default=default_config.get("parameters").get("weight_decay").get("values")[-1],
                           help='random seed')
    
    args = argparser.parse_args()
    vars(default_config).update(vars(args))
    return



def train(config=None):
    
    torch.manual_seed(default_config.get("parameters").get("seed").get("value"))
    
    run = wandb.init(
                project="h2o-ai-predict-the-llm-kaggle-competition", 
                entity=None, 
                   job_type="hyperparameter-tuning"
    )
    if "artifacts" not in os.listdir():
        raw_data_at = run.use_artifact('mustafakeser/h2o-ai-predict-the-llm-kaggle-competition/detect_llm_raw_data:v1', 
                                                       type='raw_data')
        artifact_di = raw_data_at.download()
    else: pass
    train , test = read_data()
    dataset_train, dataset_test = preprocess(train=train,test=test)
    config = wandb.config
    split_train_dataset,split_eval_dataset , tokenized_test , tokenizer = dataset_tokenize_n_split(train,dataset_train, dataset_test,config.architecture)

    
    
    
    model = create_model(model_name =config.architecture ,num_labels = 7)
    
    num_train_epochs=2.
    training_args = TrainingArguments(                                

                                output_dir='h2o-ai-sweeps',
                                report_to='wandb',  # Turn on Weights & Biases logging
                                num_train_epochs=config.epochs,
                                learning_rate=config.learning_rate,
                                lr_scheduler_type = config.learning_sch,
                                per_device_train_batch_size=config.batch_size,
                                per_device_eval_batch_size=16,
                                save_strategy='epoch',
                                evaluation_strategy='epoch',
                                logging_strategy='epoch',
                                metric_for_best_model="eval_loss", 
                                load_best_model_at_end=True,
                                remove_unused_columns=False,
                                greater_is_better=False,
                                weight_decay = config.weight_decay
                                

                                 )
    early_stopping = EarlyStoppingCallback(early_stopping_patience=2)
    trainer = Trainer(
                        model=model,
                        args=training_args,
                        train_dataset=split_train_dataset,
                        eval_dataset=split_eval_dataset,
                        callbacks = [early_stopping],
                        tokenizer=tokenizer,
        )
    trainer.train()

    
# if __name__=="__main__":
# #     wandb.agent(sweep_id, train, count=20)
#     parse_args()
#     train(default_config)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<a id="4"></a>
# <div style="border: 2px solid #555; color:black; border-radius: 10px; background-color: #0074D9; padding: 10px; font-size: 20px; text-align: center;">Sweeps</div>
* [return top](#1)

In [3]:
wandb.login(relogin=True)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
sweep_id = wandb.sweep(default_config, project='h2o-ai-predict-the-llm-kaggle-competition')

Create sweep with ID: q9q9767w
Sweep URL: https://wandb.ai/mustafakeser/h2o-ai-predict-the-llm-kaggle-competition/sweeps/q9q9767w


In [5]:
wandb.agent(sweep_id, train, count=20)

wandb: Agent Starting Run: vdaoah93 with config:
wandb: 	architecture: bert-base-uncased
wandb: 	batch_size: 4
wandb: 	epochs: 2
wandb: 	learning_rate: 0.0004682687272526259
wandb: 	learning_sch: polynomial
wandb: 	seed: 42
wandb: 	weight_decay: 0.2
wandb: Currently logged in as: mustafakeser. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb:   4 of 4 files downloaded.  


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Fold : 0
shape train : (3578,)
shape val : (398,)


...

Fold : 0
shape train : (3578,)
shape val : (398,)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'seed' was locked by 'sweep' (ignored update).
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.033500,1.964422
2,1.981200,1.946555


wandb: WARNING Source type is set to 'artifact' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/loss,█▁
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁███
train/global_step,▁▁███
train/learning_rate,█▁
train/loss,█▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [6]:
wandb.finish()

In [ ]:
#1.5 h p100